# Advanced Laboratory (MOD B)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from DataFormat import RootFileFormat, DataHistogram
from read_data import read_data

import glob

## Constants

In [2]:
# path to data files
DATA_PATH = "./data/"

CALIBRATIONS = ("137Cs22Na", "60Co", "137Cs", "22Na", "133Ba")

## Functions

In [3]:
def create_datasets(file_name):
    a_energy, a_time, b_energy, b_time, c_energy, c_time, d_energy, d_time, e_energy, e_time = read_data(file_name, RootFileFormat())
    a = DataHistogram(energy=a_energy, time=a_time)
    b = DataHistogram(energy=b_energy, time=b_time)
    c = DataHistogram(energy=c_energy, time=c_time)
    d = DataHistogram(energy=d_energy, time=d_time)
    e = DataHistogram(energy=e_energy, time=e_time)
    return a, b, c, d, e

## Data

In [4]:
calib_name = CALIBRATIONS[0]
file_name  = glob.glob(f"{DATA_PATH}/*{calib_name}*")[0]
print(file_name)

./data/HcompassF_137Cs22Na_calibration_20221205_164235.root


In [5]:
a, b, c, d, e = create_datasets(file_name)

## Exploratory Data Analysis